In [14]:
import osmnx as ox
import folium
import pandas as pd
import geopandas as gpd
from shapely import wkt 
from shapely.geometry import Polygon, MultiPolygon
import osmnx as ox
import geopandas as gpd
import json

def buscar_parques_jardines_y_calles():
    lugar = "Burgos, Spain"

    parques_y_jardines = ox.features_from_place(lugar, {
        'leisure': ['park', 'nature_reserve', 'dog_park'],
        'landuse': ['forest', "village_green", "grass"],
        'natural': ['scrub', 'heath']
    })

    jardines = ox.features_from_place(lugar, {'leisure': 'garden'})

    jardines_con_nombre = jardines[jardines['name'].notna()]

    parques_y_jardines = gpd.GeoDataFrame(pd.concat([parques_y_jardines, jardines_con_nombre], ignore_index=True))
    
    calles = ox.features_from_place(lugar, {
        'highway': ['footway', 'path', 'pedestrian', 'living_street', 'track', 'primary', 'secondary', 'tertiary', 'residential', ]
    })

    return parques_y_jardines, calles

def agregar_columna_tipo_y_id(dataframe, tipo_columna, es_calle=False):
    if es_calle:
        dataframe['tipo_calle'] = dataframe['highway']
    else:
        dataframe[tipo_columna] = dataframe.apply(lambda x: x['leisure'] if pd.notna(x['leisure']) else 
                                                (x['landuse'] if pd.notna(x['landuse']) else x['natural']), axis=1)
    dataframe['id'] = range(1, len(dataframe) + 1)
    print(f"Columnas después de agregar '{tipo_columna}': {dataframe.columns}")  
    
    dataframe['area'] = dataframe.to_crs(epsg=3857).area  
    dataframe = dataframe.to_crs(epsg=4326)  
    return dataframe

def convertir_a_string_si_es_necesario(valor):
    if isinstance(valor, (list, dict)):
        return json.dumps(valor)
    return valor

def preparar_para_geojson(dataframe):
    for columna in dataframe.columns:
        if columna != 'geometry':  
            dataframe[columna] = dataframe[columna].apply(convertir_a_string_si_es_necesario)
    return dataframe

def guardar_dataframes_en_csv_y_geojson(parques_y_jardines, calles):
    parques_y_jardines_preparados = preparar_para_geojson(parques_y_jardines.copy())
    calles_preparados = preparar_para_geojson(calles.copy())

    parques_y_jardines_preparados.to_csv('parques_y_jardines.csv', index=False)
    calles_preparados.to_csv('calles.csv', index=False)

    parques_y_jardines_preparados.to_file('parques_y_jardines.geojson', driver='GeoJSON')
    calles_preparados.to_file('calles.geojson', driver='GeoJSON')

def ajustar_poligonos_superpuestos(dataframe):
    
    poligonos_ajustados = []

    for i, area1 in dataframe.iterrows():
        poligono_ajustado = area1.geometry
        for _, area2 in dataframe.iterrows():
            if area1.id != area2.id and area1.geometry.intersects(area2.geometry):
                poligono_ajustado = poligono_ajustado.difference(area2.geometry)

        if isinstance(poligono_ajustado, (Polygon, MultiPolygon)):
            poligonos_ajustados.append(poligono_ajustado)
        else:
            poligonos_ajustados.append(area1.geometry)  
    return poligonos_ajustados

def agregar_poligonos_al_mapa(mapa, dataframe, color, opacidad):
    for area in dataframe.itertuples():
        if not isinstance(area.geometry, (Polygon, MultiPolygon)) or area.geometry.is_empty:
            continue

        nombre = getattr(area, 'name', 'Sin nombre')
        popup_text = f"ID: {area.id}<br>Nombre: {nombre}<br>Área: {area.area:.2f} m²"

        popup = folium.Popup(popup_text, parse_html=True)
        geojson = folium.GeoJson(
            area.geometry.__geo_interface__,
            style_function=lambda feature: {
                'fillColor': color,
                'color': color,
                'weight': 1,
                'fillOpacity': opacidad
            }
        )
        geojson.add_child(popup)
        geojson.add_to(mapa)

def agregar_lineas_al_mapa(mapa, dataframe, color):
    for _, calle in dataframe.iterrows():
        if calle.geometry is None or calle.geometry.is_empty:
            continue

        if calle.geometry.geom_type == 'LineString' or calle.geometry.geom_type == 'MultiLineString':
            folium.GeoJson(
                calle.geometry.__geo_interface__,
                style_function=lambda feature: {
                    'color': color,
                    'weight': 2
                }
            ).add_to(mapa)

def crear_mapa_con_elementos(dataframe, tipo_elemento, color, opacidad=None):
    mapa = folium.Map(location=[42.3439, -3.6969], zoom_start=13)

    if tipo_elemento == 'poligono':
        if 'tipo_jardin' in dataframe.columns:
            non_dog_parks = dataframe[dataframe['tipo_jardin'] != 'dog_park']
            agregar_poligonos_al_mapa(mapa, non_dog_parks, color, opacidad)

            dog_parks = dataframe[dataframe['tipo_jardin'] == 'dog_park']
            for _, dog_park in dog_parks.iterrows():
                if dog_park.geometry.geom_type in ['Polygon', 'MultiPolygon']:
                    popup_text = f"ID: {dog_park['id']}<br>Parque Canino: {dog_park.get('name', 'Sin nombre')}"
                    popup = folium.Popup(popup_text, parse_html=True)
                    folium.GeoJson(
                        dog_park.geometry.__geo_interface__,
                        style_function=lambda feature: {
                            'fillColor': 'red',  
                            'color': 'red',
                            'weight': 2,
                            'fillOpacity': 0.6}
                    ).add_child(popup).add_to(mapa)
        else:
            agregar_poligonos_al_mapa(mapa, dataframe, color, opacidad)

    elif tipo_elemento == 'linea':
        agregar_lineas_al_mapa(mapa, dataframe, color)
    return mapa

parques_y_jardines, calles = buscar_parques_jardines_y_calles()

parques_y_jardines_gdf = gpd.GeoDataFrame(parques_y_jardines, crs="EPSG:4326")
calles_gdf = gpd.GeoDataFrame(calles, crs="EPSG:4326")

parques_y_jardines_gdf = agregar_columna_tipo_y_id(parques_y_jardines_gdf, 'tipo_jardin')
calles_gdf = agregar_columna_tipo_y_id(calles_gdf, 'tipo_calle', es_calle=True)

guardar_dataframes_en_csv_y_geojson(parques_y_jardines_gdf, calles_gdf)

mapa_parques = crear_mapa_con_elementos(parques_y_jardines_gdf, 'poligono', 'green', 0.4)
mapa_parques.save('mapa_parques.html')

mapa_calles = crear_mapa_con_elementos(calles_gdf, 'linea', 'blue')
mapa_calles.save('mapa_calles.html')


Columnas después de agregar 'tipo_jardin': Index(['barrier', 'geometry', 'access', 'natural', 'source', 'name',
       'addr:city', 'addr:postcode', 'addr:street', 'operator',
       'operator:type', 'website', 'wheelchair', 'wikidata', 'wikipedia',
       'leisure', 'nodes', 'landuse', 'leaf_type', 'designation', 'name:1905',
       'name:1944', 'description', 'produce', 'place', 'surface', 'loc_name',
       'tourism', 'old_name', 'alt_name', 'type', 'grass', 'start_date',
       'phone', 'material', 'wall', 'ways', 'fee', 'source:date',
       'garden:type', 'building', 'traffic_calming', 'wikimedia_commons',
       'area', 'tipo_jardin', 'id'],
      dtype='object')
Columnas después de agregar 'tipo_calle': Index(['button_operated', 'crossing', 'highway', 'geometry', 'bus', 'name',
       'tactile_paving', 'traffic_signals:sound', 'traffic_signals:vibration',
       'crossing:island',
       ...
       'parking:condition:left:time_interval',
       'parking:condition:right:maxstay'

In [12]:
def agregar_lineas_sin_nombre_al_mapa(mapa, dataframe, color):
    for _, calle in dataframe.iterrows():
        if calle.geometry is None or calle.geometry.is_empty or pd.notna(calle.get('name')):
            continue

        if calle.geometry.geom_type in ['LineString', 'MultiLineString']:
            folium.GeoJson(
                calle.geometry.__geo_interface__,
                style_function=lambda feature: {
                    'color': color,
                    'weight': 2
                }
            ).add_to(mapa)

def crear_mapa_calles_sin_nombre(dataframe, color):
    mapa = folium.Map(location=[42.3439, -3.6969], zoom_start=13)
    agregar_lineas_sin_nombre_al_mapa(mapa, dataframe, color)
    return mapa

calles_sin_nombre_gdf = calles_gdf[calles_gdf['name'].isna()]

mapa_calles_sin_nombre = crear_mapa_calles_sin_nombre(calles_sin_nombre_gdf, 'blue')
mapa_calles_sin_nombre.save('mapa_calles_sin_nombre.html')


In [13]:
def agregar_lineas_con_nombre_al_mapa(mapa, dataframe, color):
    for _, calle in dataframe.iterrows():
        if calle.geometry is None or calle.geometry.is_empty or pd.isna(calle.get('name')):
            continue

        if calle.geometry.geom_type in ['LineString', 'MultiLineString']:
            popup = folium.Popup(calle['name'], parse_html=True)
            folium.GeoJson(
                calle.geometry.__geo_interface__,
                style_function=lambda feature: {
                    'color': color,
                    'weight': 2
                }
            ).add_child(popup).add_to(mapa)

def crear_mapa_calles_con_nombre(dataframe, color):
    mapa = folium.Map(location=[42.3439, -3.6969], zoom_start=13)
    agregar_lineas_con_nombre_al_mapa(mapa, dataframe, color)
    return mapa

calles_con_nombre_gdf = calles_gdf[calles_gdf['name'].notna()]

mapa_calles_con_nombre = crear_mapa_calles_con_nombre(calles_con_nombre_gdf, 'green')
mapa_calles_con_nombre.save('mapa_calles_con_nombre.html')
